# Referências cruzadas para exemplos em outros capítulos

O Asciidoctor não numera exemplos no formato `12.3` onde `12` é o número do capítulo e `3` é o número do exemplo no capítulo. Existe um [bug](https://github.com/asciidoctor/asciidoctor-pdf/issues/188) e uma [discussão](https://asciidoctor.zulipchat.com/#narrow/channel/279642-users/topic/Example.20numbering.20in.20chapter-example.20format/with/521557604) a respeito.

Atualmente há duas formas de numerar exemplos, figuras, etc: numeração única do início ao fim, ou reiniciar a numeração em cada capítulo.

No primeiro caso, a numeração fica frágil: se um exemplo for separado em dois para facilitar a paginação do livro impresso, isso vai mudar a numeração de todos os exemplos seguintes até o final do livro. No segundo caso, as referências geradas ficam ambíguas: se eu cito o exemplo 2 do capítulo 3 no capítulo 4, o texto gerado aparece apenas como "Exemplo 2".

Quando as referências cruzam os volumes, eu já tive que resolver este problema com um script Python, pois neste caso o Asciidoctor nem tem como gerar o texto ambíguo, ele deixa uma referência crua, como `ex_vector2d`. Meu script inclui o número do capítulo e do volume: "Exemplo 2 do Capítulo 3 (vol.1)".

Neste notebook vou esboçar uma solução para o caso dos exemplos dentro do mesmo volume, que será útil tanto no livro impresso como nas versões eletrônicas.

In [1]:
import os
import subprocess
from pathlib import Path

def find_git_root():
    path = Path(os.path.abspath(''))
    while path != path.parent:
        if (path / '.git').is_dir():
            return path
        path = path.parent
    raise LookupError(f'no .git dir found in {path} or parents')

GIT_ROOT = find_git_root()

INVALID_MSG = 'asciidoctor: INFO: possible invalid reference: '

def list_invalid_xrefs(ch: int) -> list[str]:
    adoc = GIT_ROOT / f'online/cap{ch:02d}.adoc'
    cmd = f'''asciidoctor -v {adoc} -o lixo'''
    result = subprocess.run(cmd, shell=True, stderr=subprocess.PIPE, text=True)
    seen = set()
    xrefs = []
    for line in result.stderr.splitlines():
        assert line.startswith(INVALID_MSG), '? msg: ' + line
        xref = line[len(INVALID_MSG):].strip()
        if xref not in seen:
            xrefs.append(xref)
            seen.add(xref)

    return xrefs
    

In [2]:
list_invalid_xrefs(11)

['ch_data_model',
 'ch_seq_methods',
 'ex_vector2d',
 'ch_op_overload',
 'ch_generators',
 'arrays_sec',
 'memoryview_sec',
 'what_is_hashable_sec',
 'ch_dynamic_attrs',
 'keyword_class_patterns_sec',
 'conseq_dict_internal_sec',
 'caching_properties_sec',
 'del_sec',
 'numpy_sec',
 'dataclass_further_sec',
 'ch_dataclass',
 'slice_aware_sec']

In [3]:
def ch_id(s):
    return s.startswith('ch_')

def sec_id(s):
    return s.endswith('_sec')

In [4]:
def list_possible_examples(ch):
    return [ident for ident in list_invalid_xrefs(ch) if not ch_id(ident) and not sec_id(ident)]

In [5]:
tuple(list_possible_examples(i) for i in (9, 10))

([], [])

In [6]:
list_possible_examples(11)  # capítulo revisado

['ex_vector2d']

In [7]:
list_possible_examples(12)  # capítulo revisado

['ex_vector2d_v0',
 'ex_vector2d_v1',
 'ex_pythonic_deck',
 'ex_vector2d_v3',
 'metaprog_part',
 'ex_vector2d_v3_hash']

In [8]:
list_possible_examples(13)  # capítulo revisado

['ex_pythonic_deck',
 'mapping_uml',
 'set_uml',
 'ex_bingo_callable',
 'ex_vector2d_v3_full',
 'ex_top_protocol_test']

In [9]:
list_possible_examples(14)  # capítulo revisado

['ex_strkeydict', 'waterfowl_essay']

In [10]:
list_possible_examples(15)  # capítulo revisado

['ex_tcp_mojifinder_main',
 'ex_checked_class_top',
 'ex_tombola_abc',
 'ex_lotto',
 'ex_randompick_protocol',
 'ex_primes_procs_top']

In [11]:
list_possible_examples(16)  # capítulo revisado

['ex_vector_v5',
 'ex_vector2d',
 'ex_vector2d_v0',
 'ex_vector2d_v3_full',
 'ex_tombola_bingo',
 'ex_tombola_abc']